<a href="https://colab.research.google.com/github/Medouchene/Deep-Learning/blob/main/Perceptron_logistic-regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Perceptron

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# fct prediction
def predict(row, weights):
    z = weights[0]  # Biais
    for i in range(len(row) - 1):
        z += weights[i + 1] * row[i]
    return 1 if z >= 0 else -1

In [ ]:
x = [2, 3, 2, 9, 8, 7]
w = [9, 2, 3, 2, 9, 8, 7]
print(predict(x, w))

1


In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/M2/ML/Supervised/PREP/TP2/sonar.csv")
print('HEAD :')
print(df.head()) #afficher

HEAD :
     0.02  0.0371  0.0428  0.0207  0.0954  0.0986  0.1539  0.1601  0.3109  \
0  0.0453  0.0523  0.0843  0.0689  0.1183  0.2583  0.2156  0.3481  0.3337   
1  0.0262  0.0582  0.1099  0.1083  0.0974  0.2280  0.2431  0.3771  0.5598   
2  0.0100  0.0171  0.0623  0.0205  0.0205  0.0368  0.1098  0.1276  0.0598   
3  0.0762  0.0666  0.0481  0.0394  0.0590  0.0649  0.1209  0.2467  0.3564   
4  0.0286  0.0453  0.0277  0.0174  0.0384  0.0990  0.1201  0.1833  0.2105   

   0.2111  ...  0.0027  0.0065  0.0159  0.0072  0.0167   0.018  0.0084  \
0  0.2872  ...  0.0084  0.0089  0.0048  0.0094  0.0191  0.0140  0.0049   
1  0.6194  ...  0.0232  0.0166  0.0095  0.0180  0.0244  0.0316  0.0164   
2  0.1264  ...  0.0121  0.0036  0.0150  0.0085  0.0073  0.0050  0.0044   
3  0.4459  ...  0.0031  0.0054  0.0105  0.0110  0.0015  0.0072  0.0048   
4  0.3039  ...  0.0045  0.0014  0.0038  0.0013  0.0089  0.0057  0.0027   

    0.009  0.0032  1.0  
0  0.0052  0.0044  1.0  
1  0.0095  0.0078  1.0  
2  0.0040 

In [ ]:
dataset = df.values

for row in dataset:
    if row[-1] == 0:
      row[-1] = -1

In [ ]:
# Suppose que la dernière colonne est l'étiquette
X = df.iloc[:, :-1].values  # Caractéristiques
y = df.iloc[:, -1].values   # Étiquettes
weights = np.zeros(X.shape[1] + 1)

def evaluate(X, y, weights):
    correct_predictions = 0
    for i in range(len(X)):
        prediction = predict(X[i], weights)
        if prediction == y[i]:
            correct_predictions += 1
    accuracy = correct_predictions / len(y)
    return accuracy

accuracy = evaluate(X, y, weights)
print(f'Précision : {accuracy * 100:.2f}%')

Précision : 46.38%


Logistic Regression

In [ ]:
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/M2/ML/Supervised/PREP/TP2/banking.csv")
data = data.dropna()
print(data.head())

   age          job  marital          education  default housing loan  \
0   44  blue-collar  married           basic.4y  unknown     yes   no   
1   53   technician  married            unknown       no      no   no   
2   28   management   single  university.degree       no     yes   no   
3   39     services  married        high.school       no      no   no   
4   55      retired  married           basic.4y       no     yes   no   

    contact month day_of_week  ...  campaign  pdays  previous     poutcome  \
0  cellular   aug         thu  ...         1    999         0  nonexistent   
1  cellular   nov         fri  ...         1    999         0  nonexistent   
2  cellular   jun         thu  ...         3      6         2      success   
3  cellular   apr         fri  ...         2    999         0  nonexistent   
4  cellular   aug         fri  ...         1      3         1      success   

  emp_var_rate  cons_price_idx  cons_conf_idx  euribor3m  nr_employed  y  
0          1.4   

In [ ]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp_var_rate    41188 non-null  float64
 16  cons_price_idx  41188 non-null  float64
 17  cons_conf_idx   41188 non-null 

In [ ]:
categorical_var = ["job", "marital", "education", "default", "housing", "loan", "contact", "month",
                   "day_of_week", "poutcome"]
data_new = data.drop(columns=categorical_var)

print(data_new.head())

   age  duration  campaign  pdays  previous  emp_var_rate  cons_price_idx  \
0   44       210         1    999         0           1.4          93.444   
1   53       138         1    999         0          -0.1          93.200   
2   28       339         3      6         2          -1.7          94.055   
3   39       185         2    999         0          -1.8          93.075   
4   55       137         1      3         1          -2.9          92.201   

   cons_conf_idx  euribor3m  nr_employed  y  
0          -36.1      4.963       5228.1  0  
1          -42.0      4.021       5195.8  0  
2          -39.8      0.729       4991.6  1  
3          -47.1      1.405       5099.1  0  
4          -31.4      0.869       5076.2  1  


In [ ]:
my_log = linear_model.LogisticRegression()


In [ ]:
data_X = data_new.iloc[:, 0:10]
data_Y = data_new['y']
print("Data Y : ", data_Y)

Data Y :  0        0
1        0
2        1
3        0
4        1
        ..
41183    0
41184    0
41185    0
41186    0
41187    0
Name: y, Length: 41188, dtype: int64


In [ ]:
# Normalize the feature data, falsafa zayda
# scaler = StandardScaler()
# data_X_scaled = scaler.fit_transform(data_X)"""

my_log = linear_model.LogisticRegression()

my_log.fit(data_X, data_Y)

print("Model coefficients:", my_log.coef_) # the output is coeffs wi

print("Model intercep :", my_log.intercept_) # P(Y=1/X=x) = 1/(1 + exp(-(w0 +w1x1+ .. )));

predictions = my_log.predict(data_X)
print("Output predictions:", predictions)

score = my_log.score(data_X, data_Y)
print("Output score (accuracy):", score)

Model coefficients: [[ 0.00166099  0.00454397 -0.00847747 -0.00159375 -0.03018196 -0.23634302
   0.30504207  0.04473127 -0.26021316 -0.0055078 ]]
Model intercep : [0.00314879]
Output predictions: [0 0 1 ... 0 0 0]
Output score (accuracy): 0.9092939691172186


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Hold out evaluation

In [ ]:
# Diviser les données en ensembles d'entraînement et de test (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(data_X, data_Y, test_size=0.3, random_state=42)

# Initialiser le modèle de régression logistique
my_log = linear_model.LogisticRegression()
my_log.fit(X_train, y_train)  # Entraîner le modèle

# Prédire les valeurs sur l'ensemble de test
y_pred = my_log.predict(X_test)

# Évaluer le modèle
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy du modèle:", accuracy)
print("Rapport de classification:\n", report)

Accuracy du modèle: 0.9043457149793639
Rapport de classification:
               precision    recall  f1-score   support

           0       0.93      0.97      0.95     10931
           1       0.64      0.40      0.49      1426

    accuracy                           0.90     12357
   macro avg       0.78      0.69      0.72     12357
weighted avg       0.89      0.90      0.89     12357



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Cross Validation

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

# Initialiser KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # 5 folds, mélanger les données

# Stocker les scores
scores = []

# Boucle sur les folds
for train_index, test_index in kf.split(data_X):
    X_train, X_test = data_X.iloc[train_index], data_X.iloc[test_index]  # Utiliser iloc pour accéder aux indices
    y_train, y_test = data_Y.iloc[train_index], data_Y.iloc[test_index]  # Utiliser iloc pour accéder aux indices

    # Créer et entraîner le modèle
    my_log = LogisticRegression(max_iter=1000)  # Assurez-vous d'ajuster max_iter si nécessaire
    my_log.fit(X_train, y_train)

    # Prédire les valeurs
    y_pred = my_log.predict(X_test)

    # Évaluer le modèle
    score = accuracy_score(y_test, y_pred)
    scores.append(score)

# Afficher les résultats
print("Scores de chaque fold:", scores)
print("Moyenne de l'accuracy:", np.mean(scores))
print("Écart-type de l'accuracy:", np.std(scores))


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Scores de chaque fold: [0.9067734887108522, 0.9066521000242778, 0.9119932022335518, 0.9106470802476629, 0.9102828699769333]
Moyenne de l'accuracy: 0.9092697482386557
Écart-type de l'accuracy: 0.00216444240331353


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
